In [22]:

from unet.shared import *
from unet.dataset_preparation import *


torch.manual_seed(2022)  # Setting random seed so that augmentations can be reproduced.

# Inputs for the helper functions
# Image and model params
#img_dir = '/home/users/mcl66/SeaIce/tiled512/'
img_dir = '/mnt/g/Shared drives/2021-gtc-sea-ice/trainingdata/tiled512/train/'
img_type = 'sar'
model_type = "pretrained"
workers = 8
return_type = 'values'

validation_percent = 0.1
batch_size = 1

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Create dataset
img_list = create_npy_list(training_tiles[1], img_type)

if img_type == "sar":
    is_single_band = True
    n_channels = 1
elif img_type == "modis":
    is_single_band = False
    n_channels = 3

#dataset = CustomImageDataset(img_list, single_channel, "values")

#val_percent=0.1
#_, _, train_loader_2, val_loader = split_data(dataset, val_percent, batch_size, 1)

# Loader
img_list = create_npy_list(img_dir, img_type)

train_img_list, val_img_list, n_train, n_val = split_img_list(img_list, validation_percent)

train_dataset = CustomImageAugmentDataset(train_img_list, is_single_band, return_type, True)
validation_dataset = CustomImageAugmentDataset(val_img_list, is_single_band, return_type, False)

train_loader, val_loader = create_dataloaders(train_dataset, validation_dataset, batch_size, workers)


loss = smp.utils.losses.DiceLoss()
metrics = [smp.utils.metrics.IoU(threshold=0.5),]
# Can also specify number of UNet steps and channel numbers.
model = smp.Unet(encoder_name='resnet34', encoder_weights='imagenet', decoder_use_batchnorm=True,
                 decoder_attention_type=None, in_channels=n_channels, classes=1, encoder_depth=5)
model = model.double()

optimizer = torch.optim.Adam([dict(params=model.parameters(), lr=0.0001),])

train_epoch = smp.utils.train.TrainEpoch(
    model,
    loss=loss,
    metrics=metrics,
    optimizer=optimizer,
    verbose=True,
    device=processor
)

valid_epoch = smp.utils.train.ValidEpoch(
    model,
    loss=loss,
    metrics=metrics,
    verbose=True,
    device=processor
)

# train model for n epochs

max_score = 0
n_epochs = 1

if __name__ == '__main__':
    for i in range(0, n_epochs):

        print('\nEpoch: {}'.format(i))
        
        train_logs = train_epoch.run(train_loader)
        valid_logs = valid_epoch.run(val_loader)

        # do something (save model, change lr, etc.)
        if max_score < valid_logs['iou_score']:
            max_score = valid_logs['iou_score']
            dir_checkpoint = create_checkpoint_dir(path_checkpoint, img_type, model_type='pretrained')
            torch.save(model, str(dir_checkpoint + 'best_model.pth'))
            print('Model saved!')

        if i == n_epochs/2:
            optimizer.param_groups[0]['lr'] = 1e-5
            print('Decrease decoder learning rate to 1e-5!')



Epoch: 0
train:   0%|                                 | 2/5008 [00:09<6:34:23,  4.73s/it, dice_loss - 0.6224, iou_score - 0.3177]


Traceback (most recent call last):
  File "/home/mlisaius/miniconda3/envs/unetenv/lib/python3.7/multiprocessing/queues.py", line 242, in _feed
    send_bytes(obj)
  File "/home/mlisaius/miniconda3/envs/unetenv/lib/python3.7/multiprocessing/connection.py", line 200, in send_bytes
    self._send_bytes(m[offset:offset + size])
Traceback (most recent call last):
  File "/home/mlisaius/miniconda3/envs/unetenv/lib/python3.7/multiprocessing/queues.py", line 242, in _feed
    send_bytes(obj)
  File "/home/mlisaius/miniconda3/envs/unetenv/lib/python3.7/multiprocessing/connection.py", line 200, in send_bytes
    self._send_bytes(m[offset:offset + size])
  File "/home/mlisaius/miniconda3/envs/unetenv/lib/python3.7/multiprocessing/connection.py", line 404, in _send_bytes
    self._send(header + buf)
  File "/home/mlisaius/miniconda3/envs/unetenv/lib/python3.7/multiprocessing/connection.py", line 368, in _send
    n = write(self._handle, buf)
BrokenPipeError: [Errno 32] Broken pipe
  File "/home/ml

KeyboardInterrupt: 

In [15]:
print(len(train_loader))

5008


In [20]:
print(len(train_loader_2))

5008


In [17]:
for test_images in train_loader:
    print(test_images)

{'image': tensor([[[[601., 580., 648.,  ..., 632., 590., 562.],
          [588., 526., 618.,  ..., 569., 576., 606.],
          [596., 510., 601.,  ..., 608., 615., 658.],
          ...,
          [522., 511., 563.,  ..., 716., 727., 703.],
          [547., 546., 547.,  ..., 721., 721., 709.],
          [527., 513., 505.,  ..., 695., 680., 653.]]]], dtype=torch.float64), 'mask': tensor([[[1., 1., 1.,  ..., 1., 1., 1.],
         [1., 1., 1.,  ..., 1., 1., 1.],
         [1., 1., 1.,  ..., 1., 1., 1.],
         ...,
         [1., 1., 1.,  ..., 1., 1., 1.],
         [1., 1., 1.,  ..., 1., 1., 1.],
         [1., 1., 1.,  ..., 1., 1., 1.]]], dtype=torch.float64)}
{'image': tensor([[[[409., 403., 425.,  ..., 357., 408., 438.],
          [443., 424., 401.,  ..., 414., 420., 418.],
          [451., 430., 392.,  ..., 434., 385., 388.],
          ...,
          [595., 552., 508.,  ..., 391., 392., 351.],
          [574., 567., 517.,  ..., 411., 390., 361.],
          [532., 548., 510.,  ..., 355.

{'image': tensor([[[[ 517.,  555.,  578.,  ...,  784.,  749.,  778.],
          [ 459.,  566.,  645.,  ...,  849.,  762.,  735.],
          [ 474.,  582.,  665.,  ...,  833.,  711.,  677.],
          ...,
          [ 390.,  413.,  430.,  ..., 1555., 1395., 1257.],
          [ 409.,  479.,  456.,  ..., 1687., 1397., 1140.],
          [ 463.,  467.,  468.,  ..., 1669., 1455., 1004.]]]],
       dtype=torch.float64), 'mask': tensor([[[1., 1., 1.,  ..., 1., 1., 1.],
         [1., 1., 1.,  ..., 1., 1., 1.],
         [1., 1., 1.,  ..., 1., 1., 1.],
         ...,
         [1., 1., 1.,  ..., 1., 1., 1.],
         [1., 1., 1.,  ..., 1., 1., 1.],
         [1., 1., 1.,  ..., 1., 1., 1.]]], dtype=torch.float64)}
{'image': tensor([[[[107.,  86.,  82.,  ..., 195., 177., 167.],
          [109., 108., 109.,  ..., 243., 204., 174.],
          [104., 117., 123.,  ..., 225., 198., 170.],
          ...,
          [140., 155., 175.,  ..., 240., 319., 378.],
          [157., 159., 143.,  ..., 229., 317., 363

{'image': tensor([[[[477., 434., 333.,  ..., 363., 411., 452.],
          [499., 499., 503.,  ..., 347., 405., 455.],
          [486., 525., 546.,  ..., 364., 420., 508.],
          ...,
          [427., 549., 633.,  ..., 466., 467., 444.],
          [393., 495., 586.,  ..., 500., 465., 424.],
          [428., 458., 559.,  ..., 580., 485., 389.]]]], dtype=torch.float64), 'mask': tensor([[[1., 1., 1.,  ..., 1., 1., 1.],
         [1., 1., 1.,  ..., 1., 1., 1.],
         [1., 1., 1.,  ..., 1., 1., 1.],
         ...,
         [1., 1., 1.,  ..., 1., 1., 1.],
         [1., 1., 1.,  ..., 1., 1., 1.],
         [1., 1., 1.,  ..., 1., 1., 1.]]], dtype=torch.float64)}
{'image': tensor([[[[278., 317., 351.,  ..., 259., 297., 300.],
          [255., 270., 291.,  ..., 306., 339., 338.],
          [242., 241., 239.,  ..., 345., 353., 314.],
          ...,
          [286., 276., 277.,  ..., 356., 320., 336.],
          [307., 310., 311.,  ..., 350., 326., 388.],
          [295., 304., 311.,  ..., 355.

{'image': tensor([[[[583., 637., 676.,  ..., 396., 425., 436.],
          [572., 643., 635.,  ..., 393., 430., 447.],
          [567., 593., 579.,  ..., 378., 393., 423.],
          ...,
          [522., 495., 443.,  ..., 239., 253., 248.],
          [566., 557., 502.,  ..., 245., 261., 255.],
          [519., 537., 519.,  ..., 263., 251., 255.]]]], dtype=torch.float64), 'mask': tensor([[[1., 1., 1.,  ..., 1., 1., 1.],
         [1., 1., 1.,  ..., 1., 1., 1.],
         [1., 1., 1.,  ..., 1., 1., 1.],
         ...,
         [1., 1., 1.,  ..., 1., 1., 1.],
         [1., 1., 1.,  ..., 1., 1., 1.],
         [1., 1., 1.,  ..., 1., 1., 1.]]], dtype=torch.float64)}
{'image': tensor([[[[183., 186., 190.,  ..., 190., 195., 180.],
          [207., 198., 197.,  ..., 180., 180., 173.],
          [250., 241., 221.,  ..., 141., 152., 163.],
          ...,
          [151., 158., 149.,  ..., 182., 169., 152.],
          [140., 153., 147.,  ..., 183., 168., 144.],
          [138., 145., 139.,  ..., 166.

{'image': tensor([[[[595., 575., 517.,  ..., 427., 412., 416.],
          [564., 547., 499.,  ..., 429., 432., 440.],
          [468., 462., 425.,  ..., 375., 421., 454.],
          ...,
          [386., 356., 369.,  ..., 137., 162., 215.],
          [400., 395., 409.,  ..., 136., 151., 203.],
          [438., 452., 443.,  ..., 145., 166., 204.]]]], dtype=torch.float64), 'mask': tensor([[[0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         ...,
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.]]], dtype=torch.float64)}
{'image': tensor([[[[674., 629., 610.,  ..., 572., 580., 571.],
          [628., 590., 591.,  ..., 546., 547., 568.],
          [548., 568., 588.,  ..., 530., 528., 573.],
          ...,
          [642., 599., 542.,  ..., 670., 611., 597.],
          [648., 578., 544.,  ..., 610., 587., 605.],
          [638., 601., 609.,  ..., 612.

{'image': tensor([[[[ 426.,  509.,  419.,  ...,  323.,  335.,  385.],
          [ 346.,  384.,  372.,  ...,  323.,  404.,  392.],
          [ 439.,  384.,  375.,  ...,  427.,  338.,  392.],
          ...,
          [ 854.,  916.,  922.,  ..., 1248., 1092., 1140.],
          [ 901.,  795.,  748.,  ..., 1453., 1092.,  839.],
          [ 701.,  904., 1020.,  ...,  909.,  878.,  740.]]]],
       dtype=torch.float64), 'mask': tensor([[[1., 1., 1.,  ..., 1., 1., 1.],
         [1., 1., 1.,  ..., 1., 1., 1.],
         [1., 1., 1.,  ..., 1., 1., 1.],
         ...,
         [1., 1., 1.,  ..., 1., 1., 1.],
         [1., 1., 1.,  ..., 1., 1., 1.],
         [1., 1., 1.,  ..., 1., 1., 1.]]], dtype=torch.float64)}
{'image': tensor([[[[4170., 5410., 6527.,  ..., 3239., 4707., 3732.],
          [7290., 6830., 5494.,  ..., 3554., 3614., 3585.],
          [5203., 4046., 4440.,  ..., 2479., 4008., 4679.],
          ...,
          [4827., 6516., 4418.,  ..., 5506., 4732., 4488.],
          [4393., 4975., 2

KeyboardInterrupt: 

In [21]:
for test_images in train_loader_2:
    print(test_images)

[tensor([[[[357., 388., 379.,  ..., 545., 527., 505.],
          [298., 327., 344.,  ..., 532., 503., 455.],
          [329., 328., 328.,  ..., 562., 516., 469.],
          ...,
          [404., 372., 323.,  ..., 688., 688., 654.],
          [363., 329., 305.,  ..., 704., 692., 648.],
          [328., 363., 383.,  ..., 714., 698., 639.]]]], dtype=torch.float64), tensor([[[[1., 1., 1.,  ..., 1., 1., 1.],
          [1., 1., 1.,  ..., 1., 1., 1.],
          [1., 1., 1.,  ..., 1., 1., 1.],
          ...,
          [1., 1., 1.,  ..., 1., 1., 1.],
          [1., 1., 1.,  ..., 1., 1., 1.],
          [1., 1., 1.,  ..., 1., 1., 1.]]]], dtype=torch.float64)]
[tensor([[[[ 428.,  443.,  431.,  ...,  764.,  867.,  924.],
          [ 440.,  443.,  426.,  ...,  767.,  857.,  924.],
          [ 459.,  433.,  411.,  ...,  778.,  860.,  906.],
          ...,
          [ 449.,  451.,  443.,  ..., 1551., 1457., 1388.],
          [ 437.,  461.,  443.,  ..., 1538., 1423., 1425.],
          [ 457.,  453.,  4

[tensor([[[[103.,  93.,  90.,  ..., 103., 107., 108.],
          [ 90.,  82.,  83.,  ...,  97., 113., 127.],
          [ 81.,  70.,  78.,  ...,  98., 107., 120.],
          ...,
          [118., 138., 131.,  ..., 100.,  96.,  92.],
          [105., 125., 123.,  ..., 109., 106., 101.],
          [ 97., 114., 114.,  ...,  98.,  92.,  93.]]]], dtype=torch.float64), tensor([[[[0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          ...,
          [1., 1., 1.,  ..., 0., 0., 0.],
          [1., 1., 1.,  ..., 0., 0., 0.],
          [1., 1., 1.,  ..., 0., 0., 0.]]]], dtype=torch.float64)]
[tensor([[[[ 744.,  647.,  659.,  ...,  948., 1011., 1098.],
          [ 758.,  676.,  621.,  ...,  934., 1000., 1100.],
          [ 804.,  743.,  724.,  ...,  980.,  992., 1096.],
          ...,
          [1249., 1225., 1043.,  ...,    0.,    0.,    0.],
          [1165., 1126.,  976.,  ...,    0.,    0.,    0.],
          [1080., 1033.,  8

Exception ignored in: <function _MultiProcessingDataLoaderIter.__del__ at 0x7fc7c1d6cd40>
Traceback (most recent call last):
  File "/home/mlisaius/miniconda3/envs/unetenv/lib/python3.7/site-packages/torch/utils/data/dataloader.py", line 1328, in __del__
    self._shutdown_workers()
  File "/home/mlisaius/miniconda3/envs/unetenv/lib/python3.7/site-packages/torch/utils/data/dataloader.py", line 1320, in _shutdown_workers
    if w.is_alive():
  File "/home/mlisaius/miniconda3/envs/unetenv/lib/python3.7/multiprocessing/process.py", line 151, in is_alive
    assert self._parent_pid == os.getpid(), 'can only test a child process'
AssertionError: can only test a child process


[tensor([[[[605., 601., 598.,  ..., 539., 480., 456.],
          [597., 548., 535.,  ..., 545., 487., 453.],
          [623., 548., 522.,  ..., 648., 648., 597.],
          ...,
          [480., 502., 534.,  ..., 986., 843., 715.],
          [580., 559., 562.,  ..., 959., 780., 740.],
          [746., 687., 695.,  ..., 824., 663., 721.]]]], dtype=torch.float64), tensor([[[[1., 1., 1.,  ..., 1., 1., 1.],
          [1., 1., 1.,  ..., 1., 1., 1.],
          [1., 1., 1.,  ..., 1., 1., 1.],
          ...,
          [1., 1., 1.,  ..., 1., 1., 1.],
          [1., 1., 1.,  ..., 1., 1., 1.],
          [1., 1., 1.,  ..., 1., 1., 1.]]]], dtype=torch.float64)]
[tensor([[[[370., 323., 222.,  ..., 254., 247., 224.],
          [322., 294., 232.,  ..., 242., 242., 235.],
          [269., 254., 236.,  ..., 231., 211., 207.],
          ...,
          [317., 306., 301.,  ..., 312., 312., 298.],
          [295., 294., 313.,  ..., 357., 347., 319.],
          [306., 343., 380.,  ..., 358., 356., 333.]]]], 

KeyboardInterrupt: 